# orography 전처리 코드
- 기준 grid_id 기준으로 결합
- elevation 보간(interpolation) 포함

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

# 경로 설정
data_dir = "orography_data"  # .nc 파일이 모두 압축 해제된 폴더
grid_path = "reference_grid_ids.csv"
output_csv = "orography_processed.csv"
output_parquet = "orography_processed.parquet"

# 기준 grid_id 불러오기
grid_df = pd.read_csv(grid_path)import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from tqdm import tqdm

# 입력 파일 경로 설정
tif_path = "global_PCT_URBAN_1km_masked_QC_v1.1.tif"
grid_path = "reference_grid_ids.csv"

# 출력 파일 경로 설정
output_csv = "urban_frac_processed.csv"
output_parquet = "urban_frac_processed.parquet"

# 기준 grid_id 로드
grid_df = pd.read_csv(grid_path)

# GeoTIFF 열기
with rasterio.open(tif_path) as src:
    band1 = src.read(1)
    transform = src.transform
    nodata = src.nodata

# raster에서 lat/lon 계산
rows, cols = np.indices(band1.shape)
xs, ys = rasterio.transform.xy(transform, rows, cols)
lons = np.array(xs)
lats = np.array(ys)
vals = band1

# 평탄화
flat_lat = lats.ravel()
flat_lon = lons.ravel()
flat_val = vals.ravel()

# 결측치 제거
mask = ~np.isnan(flat_val)
flat_lat = flat_lat[mask]
flat_lon = flat_lon[mask]
flat_val = flat_val[mask]

# grid_id 계산 함수
def compute_grid_id(lat, lon):
    lat_bin = np.floor(lat / 0.1)
    lon_bin = np.floor(lon / 0.1)
    return int((lat_bin + 900) * 3600 + (lon_bin + 1800))

# 데이터프레임 생성
urban_df = pd.DataFrame({
    "latitude": flat_lat,
    "longitude": flat_lon,
    "urban_frac": flat_val
})
urban_df["grid_id"] = urban_df.apply(lambda row: compute_grid_id(row["latitude"], row["longitude"]), axis=1)

# 중복 grid_id 평균 처리
urban_grouped = urban_df.groupby("grid_id", as_index=False)["urban_frac"].mean()

# 기준 grid와 조인
merged = grid_df.merge(urban_grouped, on="grid_id", how="left")

# 누락값 보간
missing_before = merged["urban_frac"].isna().sum()
merged["urban_frac"] = merged["urban_frac"].interpolate(method="linear")
missing_after = merged["urban_frac"].isna().sum()

# 저장
merged.to_csv(output_csv, index=False)
merged.to_parquet(output_parquet, index=False)

print("저장 완료:", output_csv, output_parquet)
print("보간 전 결측값 수:", missing_before)
print("보간 후 결측값 수:", missing_after)


# grid_id 계산 함수
def compute_grid_id(lat, lon):
    lat_bin = np.floor(lat / 0.1)
    lon_bin = np.floor(lon / 0.1)
    return int((lat_bin + 900) * 3600 + (lon_bin + 1800))

# 전체 파일에서 고도 데이터 추출
all_data = []
nc_files = [f for f in os.listdir(data_dir) if f.endswith('.nc')]

for fname in tqdm(nc_files, desc="고도 데이터 처리 중"):
    try:
        ds = xr.open_dataset(os.path.join(data_dir, fname))
        elev = ds["SRTMGL1_DEM"]
        lat = elev["lat"].values
        lon = elev["lon"].values
        val = elev.values

        lon_grid, lat_grid = np.meshgrid(lon, lat)
        flat_lat = lat_grid.flatten()
        flat_lon = lon_grid.flatten()
        flat_val = val.flatten()

        mask = ~np.isnan(flat_val)
        flat_lat = flat_lat[mask]
        flat_lon = flat_lon[mask]
        flat_val = flat_val[mask]

        grid_ids = [compute_grid_id(la, lo) for la, lo in zip(flat_lat, flat_lon)]

        df = pd.DataFrame({
            "grid_id": grid_ids,
            "elevation": flat_val
        })
        all_data.append(df)
    except Exception as e:
        print(f"오류 발생: {fname} - {e}")

# 하나로 합치고 grid_id 기준 평균
elev_df = pd.concat(all_data, ignore_index=True)
elev_grouped = elev_df.groupby("grid_id", as_index=False)["elevation"].mean()

# 기준 grid_id와 조인
merged = grid_df.merge(elev_grouped, on="grid_id", how="left")

# 보간 처리
missing_before = merged["elevation"].isna().sum()
merged["elevation"] = merged["elevation"].interpolate(method="linear")
missing_after = merged["elevation"].isna().sum()

# 저장
merged.to_csv(output_csv, index=False)
merged.to_parquet(output_parquet, index=False)

# urban_frac 전처리 코드

In [ ]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from tqdm import tqdm

# 입력 파일 경로 설정
tif_path = "global_PCT_URBAN_1km_masked_QC_v1.1.tif"
grid_path = "reference_grid_ids.csv"

# 출력 파일 경로 설정
output_csv = "urban_frac_processed.csv"
output_parquet = "urban_frac_processed.parquet"

# 기준 grid_id 로드
grid_df = pd.read_csv(grid_path)

# GeoTIFF 열기
with rasterio.open(tif_path) as src:
    band1 = src.read(1)
    transform = src.transform
    nodata = src.nodata

# raster에서 lat/lon 계산
rows, cols = np.indices(band1.shape)
xs, ys = rasterio.transform.xy(transform, rows, cols)
lons = np.array(xs)
lats = np.array(ys)
vals = band1

# 평탄화
flat_lat = lats.ravel()
flat_lon = lons.ravel()
flat_val = vals.ravel()

# 결측치 제거
mask = ~np.isnan(flat_val)
flat_lat = flat_lat[mask]
flat_lon = flat_lon[mask]
flat_val = flat_val[mask]

# grid_id 계산 함수
def compute_grid_id(lat, lon):
    lat_bin = np.floor(lat / 0.1)
    lon_bin = np.floor(lon / 0.1)
    return int((lat_bin + 900) * 3600 + (lon_bin + 1800))

# 데이터프레임 생성
urban_df = pd.DataFrame({
    "latitude": flat_lat,
    "longitude": flat_lon,
    "urban_frac": flat_val
})
urban_df["grid_id"] = urban_df.apply(lambda row: compute_grid_id(row["latitude"], row["longitude"]), axis=1)

# 중복 grid_id 평균 처리
urban_grouped = urban_df.groupby("grid_id", as_index=False)["urban_frac"].mean()

# 기준 grid와 조인
merged = grid_df.merge(urban_grouped, on="grid_id", how="left")

# 누락값 보간
missing_before = merged["urban_frac"].isna().sum()
merged["urban_frac"] = merged["urban_frac"].interpolate(method="linear")
missing_after = merged["urban_frac"].isna().sum()

# 저장
merged.to_csv(output_csv, index=False)
merged.to_parquet(output_parquet, index=False)


# join_orography_urban

In [ ]:
import pandas as pd

# 파일 경로
weather_path = "weather_af_land_pop_road_vege_lai_lfmc_dfmc_fuel.parquet"
oro_path = "orography_processed.csv"
urban_path = "urban_frac_processed.csv"

# 데이터 불러오기
weather_df = pd.read_parquet(weather_path)
oro_df = pd.read_csv(oro_path)
urban_df = pd.read_csv(urban_path)

# 조인
merged = weather_df.merge(oro_df, on="grid_id", how="left")
merged = merged.merge(urban_df, on="grid_id", how="left")

# 저장
merged.to_csv("joined_orography_urban.csv", index=False)
merged.to_parquet("joined_orography_urban.parquet", index=False)
